In [1]:
import pandas as pd
import sklearn as sk

In [2]:
df_hospital = pd.read_csv('../../data/2023-01-08/base_data_prepared.csv', index_col=0)
df_zip = pd.read_csv('../../data/plz_einwohner.csv')

In [3]:
df_hospital.head()

,Name,City,Street,Beds,Cases,Phone,Portraitlink,Lat,Lon
0,Charité - Universitätsmedizin Berlin,10117 Berlin,Charitéplatz 1,3.011,1.510.450,030 45050,/app/portrait/28475032679be264/start,52.526136,13.374519
1,Universitätsmedizin der Johannes Gutenberg-Uni...,55131 Mainz,Langenbeckstraße 1,1.675,302.632,06131 17 0,/app/portrait/b985292fd83e0a2a/start,49.993535,8.255683
2,Universitätsklinikum Tübingen,72076 Tübingen,Geissweg 3,1.585,70.733,07071 29 0,/app/portrait/16d84a8154635564/start,48.527739,9.048931
3,Universitätsmedizin Göttingen,37075 Göttingen,Robert-Koch-Straße 40,1.563,274.225,0551 39 0,/app/portrait/7c0ee39a26fa8dec/start,51.549934,9.945054
4,Universitätsklinikum Freiburg,79106 Freiburg,Hugstetter Straße 55,1.554,561.570,0761 270 0,/app/portrait/bd490cfe8933289f/start,48.006648,7.838115


In [4]:
df_hospital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 0 to 2492
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          2493 non-null   object 
 1   City          2493 non-null   object 
 2   Street        2493 non-null   object 
 3   Beds          2493 non-null   float64
 4   Cases         2493 non-null   object 
 5   Phone         2476 non-null   object 
 6   Portraitlink  2493 non-null   object 
 7   Lat           2493 non-null   float64
 8   Lon           2493 non-null   float64
dtypes: float64(3), object(6)
memory usage: 194.8+ KB


In [5]:
df_nan = df_hospital[df_hospital['Lat'].isna()]

In [6]:
df_nan.head(20)

,Name,City,Street,Beds,Cases,Phone,Portraitlink,Lat,Lon


In [7]:
df_zip.head()

,plz,note,einwohner,qkm,lat,lon
0,1067,01067 Dresden,11957,6.866862,51.06019,13.71117
1,1069,01069 Dresden,25491,5.351816,51.03964,13.73030
2,1097,01097 Dresden,14821,3.297752,51.06945,13.73781
3,1099,01099 Dresden,28018,58.506789,51.09272,13.82842
4,1108,01108 Dresden,5876,16.447222,51.15180,13.79227


In [8]:
zip_coords = list(zip(df_zip['lat'], df_zip['lon']))
hospital_coords = list(zip(df_hospital['Lat'], df_hospital['Lon']))

In [15]:
from sklearn.neighbors import KDTree
results = []
# Create a k-d tree from the set of points in B
points_b = hospital_coords
tree = KDTree(points_b)

# Find the nearest neighbor for each point in A
points_a = zip_coords
for point in points_a:
    dist, index = tree.query([point], k=1)
    nearest_neighbor = points_b[index[0][0]]
    results.append([dist[0][0],index[0,0],nearest_neighbor[0], nearest_neighbor[0]])

In [16]:
df_zip[['distance2NN', 'indexNN', 'latNN', 'lonNN']] = results

In [17]:
df_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8170 entries, 0 to 8169
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   plz          8170 non-null   int64  
 1   note         8170 non-null   object 
 2   einwohner    8170 non-null   int64  
 3   qkm          8170 non-null   float64
 4   lat          8170 non-null   float64
 5   lon          8170 non-null   float64
 6   distance2NN  8170 non-null   float64
 7   indexNN      8170 non-null   float64
 8   latNN        8170 non-null   float64
 9   lonNN        8170 non-null   float64
dtypes: float64(7), int64(2), object(1)
memory usage: 638.4+ KB


In [18]:
df_zip.describe()

,plz,einwohner,qkm,lat,lon,distance2NN,indexNN,latNN,lonNN
count,8170.000000,8170.000000,8170.000000,8170.000000,8170.000000,8170.000000,8170.000000,8170.000000,8170.000000
mean,55863.201102,9831.357405,43.730761,50.554190,9.981913,0.074162,1205.309302,50.553355,50.553355
std,29345.558040,8865.672875,52.647218,1.865601,1.993551,0.051350,688.030456,1.864870,1.864870
min,1067.000000,0.000000,0.001829,47.370850,5.984790,0.000001,0.000000,47.410500,47.410500
25%,29342.750000,2789.250000,12.745523,48.979143,8.439563,0.031960,651.000000,48.957197,48.957197
50%,57868.500000,6555.000000,27.278689,50.352630,9.865060,0.067427,1194.000000,50.351150,50.351150
75%,83480.000000,15162.250000,56.223991,51.945145,11.489735,0.106243,1764.000000,51.939819,51.939819
max,99998.000000,58826.000000,891.886712,55.015310,14.979570,0.378287,2492.000000,54.925469,54.925469


In [20]:
df_zip.to_csv('../../data/df_zip_with_nearest_neighbors.csv')